In [21]:
import pandas as pd
import folium
from folium.plugins import Fullscreen
from sqlalchemy import create_engine
import utm
import branca.colormap as cm
from sklearn.ensemble import IsolationForest
from ipywidgets import interact, widgets
from IPython.display import display


In [22]:
# Detalles de la conexión
server = '192.168.200.31'
database = 'jmineops'
username = 'tinformacion'
password = 'Timeinlondon$'
conn_str = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
engine = create_engine(conn_str)

query = """
SELECT [time]
      ,[Northing] AS j
      ,[Easting] AS i
      ,[Elevation]
      ,[speed] AS Speed
      ,[full_name]
      ,[Alarma]
  FROM [jmineops].[dbo].[T_sensors_speed_coord]
"""

# Ejecutar la consulta y cargar los datos en un DataFrame
data = pd.read_sql(query, engine)

# Convertir las coordenadas UTM a latitud y longitud
def utm_to_latlon(row):
    lat, lon = utm.to_latlon(row['i'], row['j'], 13, 'N')  # Ajustar la zona y el hemisferio según sea necesario
    return pd.Series([lat, lon])

data[['latitude', 'longitude']] = data.apply(utm_to_latlon, axis=1)

# Filtrar datos para eliminar coordenadas inválidas
valid_lat_range = (18, 22)
valid_lon_range = (-107, -100)
data = data[(data['latitude'].between(*valid_lat_range)) & (data['longitude'].between(*valid_lon_range))]

# Convertir la columna 'time' a datetime
data['time'] = pd.to_datetime(data['time'])

# Eliminar filas con valores faltantes
data = data.dropna(subset=['latitude', 'longitude', 'Speed', 'Alarma', 'Elevation'])

# Filtrar filas donde la velocidad esté entre 10 y 40 km/h
data = data[(data['Speed'] > 10) & (data['Speed'] < 40)]

# Detección y eliminación de outliers
iso_forest = IsolationForest(contamination=0.01, random_state=42)
outliers = iso_forest.fit_predict(data[['latitude', 'longitude', 'Speed']])
data = data[outliers == 1]

# Muestra un subconjunto de los datos
sample_size = 5000  # Ajusta este valor según sea necesario
if len(data) > sample_size:
    data_sample = data.sample(n=sample_size, random_state=42)
else:
    data_sample = data

# Display initial data sample
data_sample.head()


,time,j,i,Elevation,Speed,full_name,Alarma,latitude,longitude
160823,2022-09-23 17:12:51,2.143978e+06,595073.129373,1164.614285,17.0,Juan Pablo Aguilar Vargas,TC Filt,19.387718,-104.094603
268635,2023-02-12 16:32:57,2.143168e+06,594108.751595,1029.654285,12.0,Marcos Gonzalo Licea Zuñiga,Hoist Screen,19.380449,-104.103826
389694,2023-12-31 08:27:36,2.143013e+06,594188.913426,969.374285,28.0,Omar González de los Santos,Engine Coolant Pump Outlet Temperature Sensor,19.379044,-104.103071
179690,2022-10-22 12:25:07,2.142133e+06,593165.501534,926.444285,16.0,Rubén Lara Virgen,High Fuel Pressure,19.371137,-104.112859
474399,2024-05-18 16:22:01,2.143479e+06,594643.829555,1040.284285,24.0,Juan Pablo Aguilar Vargas,High Left Front Brake Oil Temperature,19.383229,-104.098716


In [23]:
def plot_map(start_date, end_date, employee):
    filtered_data = data_sample[
        (data_sample['time'] >= pd.to_datetime(start_date)) &
        (data_sample['time'] <= pd.to_datetime(end_date))
    ]
    
    if employee != 'Todos':
        filtered_data = filtered_data[filtered_data['full_name'] == employee]
    
    # Mapa de velocidad
    speed_map_center = [filtered_data['latitude'].mean(), filtered_data['longitude'].mean()]
    speed_map = folium.Map(location=speed_map_center, zoom_start=15)
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Esri',
        name='Esri Satellite',
        overlay=False,
        control=True
    ).add_to(speed_map)

    Fullscreen(position='topright', title='Pantalla completa', title_cancel='Salir de pantalla completa').add_to(speed_map)

    # Agregar puntos de velocidad usando CircleMarker
    for _, row in filtered_data.iterrows():
        folium.CircleMarker(
            location=(row['latitude'], row['longitude']),
            radius=2,
            color=cm.LinearColormap(['blue', 'lime', 'red'], vmin=10, vmax=40)(row['Speed']),
            fill=True,
            fill_opacity=0.7,
            popup=f"Velocidad: {row['Speed']} km/h"
        ).add_to(speed_map)
    
    # Mostrar mapa
    display(speed_map)
    
    # Crear y mostrar barra de color
    colormap = cm.LinearColormap(['blue', 'lime', 'red'], vmin=10, vmax=40, caption='Velocidad (km/h)')
    display(colormap)

# Interact function
interact(
    plot_map,
    start_date=widgets.DatePicker(value=data['time'].min().date(), description='Fecha de inicio'),
    end_date=widgets.DatePicker(value=data['time'].max().date(), description='Fecha de fin'),
    employee=widgets.Dropdown(options=['Todos'] + list(data['full_name'].unique()), description='Empleado')
)


interactive(children=(DatePicker(value=datetime.date(2020, 1, 1), description='Fecha de inicio', step=1), Date…

<function __main__.plot_map(start_date, end_date, employee)>

In [36]:
import pandas as pd
from sqlalchemy import create_engine
import utm
import folium
from IPython.display import display
import numpy as np
from scipy.interpolate import splprep, splev
import branca.colormap as cm
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN

# Detalles de la conexión a la base de datos
server = '192.168.200.31'
database = 'jmineops'
username = 'tinformacion'
password = 'Timeinlondon$'
conn_str = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
engine = create_engine(conn_str)

# Consulta SQL para obtener los datos
query = """
SELECT [time]
      ,[Northing] AS j
      ,[Easting] AS i
      ,[Elevation]
      ,[speed] AS Speed
      ,[full_name]
      ,[Alarma]
  FROM [jmineops].[dbo].[T_sensors_speed_coord]
"""

# Ejecutar la consulta y cargar los datos en un DataFrame
data = pd.read_sql(query, engine)

# Convertir las coordenadas UTM a latitud y longitud
def utm_to_latlon(row):
    lat, lon = utm.to_latlon(row['i'], row['j'], 13, 'N')  # Ajustar la zona y el hemisferio según sea necesario
    return pd.Series([lat, lon])

data[['latitude', 'longitude']] = data.apply(utm_to_latlon, axis=1)

# Filtrar datos para eliminar coordenadas inválidas
valid_lat_range = (18, 22)
valid_lon_range = (-107, -100)
data = data[(data['latitude'].between(*valid_lat_range)) & (data['longitude'].between(*valid_lon_range))]

# Convertir la columna 'time' a datetime
data['time'] = pd.to_datetime(data['time'])

# Eliminar filas con valores faltantes
data = data.dropna(subset=['latitude', 'longitude', 'Speed', 'Alarma', 'Elevation'])

# Filtrar filas donde la velocidad esté entre 10 y 40 km/h
data = data[(data['Speed'] > 10) & (data['Speed'] < 40)]

# Detección y eliminación de outliers
iso_forest = IsolationForest(contamination=0.01, random_state=42)
outliers = iso_forest.fit_predict(data[['latitude', 'longitude', 'Speed']])
data = data[outliers == 1]

# Muestra un subconjunto de los datos
sample_size = 5000  # Ajusta este valor según sea necesario
if len(data) > sample_size:
    data_sample = data.sample(n=sample_size, random_state=42)
else:
    data_sample = data

# Utilizar DBSCAN para identificar caminos
coords = data_sample[['latitude', 'longitude']].values
db = DBSCAN(eps=0.0005, min_samples=10).fit(coords)  # Ajusta los parámetros según el caso
data_sample['cluster'] = db.labels_

# Filtrar puntos que pertenecen a un cluster válido
data_sample = data_sample[data_sample['cluster'] != -1]

# Función para eliminar coordenadas duplicadas
def remove_duplicate_coordinates(coords):
    unique_coords = np.unique(coords, axis=0)
    return unique_coords

# Suavizar la trayectoria usando Spline
def smooth_path(coords, num_points=300):
    coords = remove_duplicate_coordinates(coords)  # Eliminar duplicados
    # Verifica que las coordenadas sean adecuadas para suavizar
    if len(coords) < 4:  # Necesitamos al menos 4 puntos para una spline cúbica
        return coords
    tck, u = splprep(coords.T, s=0.2)
    unew = np.linspace(0, 1, num_points)
    smooth_coords = splev(unew, tck)
    return np.array(smooth_coords).T

# Función para crear y mostrar el mapa
def plot_smooth_trajectory():
    # Crear un mapa centrado en la ubicación promedio de los datos
    average_lat = data_sample['latitude'].mean()
    average_lon = data_sample['longitude'].mean()
    mymap = folium.Map(location=[average_lat, average_lon], zoom_start=15)

    # Agregar una capa de imagen satelital
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Esri',
        name='Esri Satellite',
        overlay=False,
        control=True
    ).add_to(mymap)

    # Crear una función para mapear la velocidad a un color
    colormap = cm.LinearColormap(['blue', 'lime', 'red'], vmin=10, vmax=40, caption='Velocidad (km/h)')
    colormap.add_to(mymap)

    # Agrupar por cluster y ordenar por tiempo para crear rutas coherentes
    for _, cluster_data in data_sample.groupby('cluster'):
        cluster_data = cluster_data.sort_values(by='time')
        coordinates = np.array(list(zip(cluster_data['latitude'], cluster_data['longitude'])))
        speeds = cluster_data['Speed'].values

        if len(coordinates) > 3:  # Asegúrate de tener suficientes puntos para suavizar
            smooth_coords = smooth_path(coordinates)

            # Dibujar líneas suaves
            for i in range(len(smooth_coords) - 1):
                folium.PolyLine(
                    [smooth_coords[i], smooth_coords[i + 1]],
                    color=colormap(speeds[i % len(speeds)]),  # Usar índice válido
                    weight=2.5,
                    opacity=0.8
                ).add_to(mymap)

    # Mostrar el mapa en el cuaderno de Jupyter
    display(mymap)

# Llamar a la función para mostrar el mapa
plot_smooth_trajectory()
